In [1]:
import os
import csv

In [2]:
from supabase import create_client, Client

In [3]:
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")

In [4]:
supabase: Client = create_client(url, key)

In [5]:
data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [32]:
def generate_rounds(rounds_csv, competition_id: int):
    rounds_insert = []
    with open(rounds_csv, mode='r', encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            row['competition_id'] = competition_id
            rounds_insert.append(row)
    return rounds_insert

def add_rounds_to_db(rounds_csv, competition_id, supabase_client):
    supabase.table("competitionround").upsert(generate_rounds(rounds_csv, competition_id), ignore_duplicates=True, on_conflict="round_number, competition_id").execute()

In [ ]:
add_rounds_to_db("rounds-2023-ncaa-tournament.csv", 1, supabase)

In [39]:
def generate_conferences(conferences_csv, league_unique: str):
    conferences_insert = []
    with open(conferences_csv, mode='r', encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            trimmed_row = {key: row[key].lower() if key == 'conference_unique' else row[key] for key in row.keys()
                           & {'conference_unique', 'conference_name'}}
            trimmed_row['league_unique'] = league_unique.lower()
            conferences_insert.append(trimmed_row)
    return conferences_insert

def add_conferences_to_db(conferences_csv, leage_unique: str, supabase_client):
    supabase.table("conference").insert(generate_conferences(conferences_csv, leage_unique)).execute()

In [ ]:
add_conferences_to_db("ncaambb-conferences-2023.csv", "ncaambb", supabase)

In [6]:
from datetime import date
def generate_teams(team_stats_csv, team_seeds_csv, league_unique: str, competition_id: int, expected_num_teams: int, thru=date.today()):
    team_insert = []
    team_competition_insert = []
    team_seed_dict = {}
    with open(team_seeds_csv, mode='r', encoding='utf-8-sig') as csv_file:
        seeds_csv_reader = csv.DictReader(csv_file)
        for row in seeds_csv_reader:
            team_seed_row = {key: row[key].lower() if key == 'team_unique' else row[key] for key in row.keys()
                             & {'team_unique', 'seed','overall_seed', 'region', 'round_started'}}
            team_seed_row['competition_id'] = competition_id
            team_seed_row['league_unique'] = league_unique
            team_seed_dict[team_seed_row['team_unique']] = team_seed_row
    if len(team_seed_dict) != expected_num_teams:
        print(team_seed_dict)
        raise ValueError("Number of teams in seed file does not match expected number of teams")
    with open(team_stats_csv, mode='r', encoding='utf-8-sig') as csv_file:
        stats_csv_reader = csv.DictReader(csv_file)
        for row in stats_csv_reader:
            team_unique = row['abbreviation'].lower()
            if team_unique[:-6] == "/women":
                team_unique = team_unique[:-6]
            if team_unique[:-4] == "/men":
                team_unique = team_unique[:-4]
            team_row = {'league_unique': league_unique,
                        'team_unique': team_unique,
                        'team_name': row['name']}
            team_insert.append(team_row)
            team_seed_dict[team_unique]['team_stats'] = {key: float(row[key]) if row[key] else 0 for key in row.keys() & {'effective_field_goal_percentage','strength_of_schedule','assist_percentage', 'free_throw_attempt_rate',  'offensive_rating', 'opp_effective_field_goal_percentage', 'two_point_field_goal_percentage', 'three_point_field_goal_percentage', 'pace', 'three_point_attempt_rate', 'true_shooting_percentage', 'turnover_percentage'}}
            team_seed_dict[team_unique]['team_win_loss'] = {key: float(row[key]) if row[key] else 0 for key in row.keys() & {'games_played','wins', 'losses','conference_wins', 'conference_losses'}}
            team_seed_dict[team_unique]['team_stats']['conference'] = row['conference'].lower()
            team_seed_dict[team_unique]['stats_thru'] = thru
            team_competition_insert.append(team_seed_dict[team_unique])
    if len(team_insert) != expected_num_teams or len(team_competition_insert) != expected_num_teams:
        print(team_seed_dict, team_insert)
        raise ValueError("Number of teams in csv does not match expected number of teams after stat processing")
    else:
        return [team_insert, team_competition_insert]


def add_teams_to_db(team_stats_csv, team_seeds_csv, leage_unique: str, competition_id: str, expected_num_teams: int, thru, supabase_client):
    [teams, team_competition] = generate_teams(team_stats_csv, team_seeds_csv, leage_unique, competition_id, expected_num_teams, thru)
    supabase.table("team").upsert(teams, ignore_duplicates=True,
                                  on_conflict="team_unique, league_unique").execute()
    supabase_client.table("team_competition").upsert(team_competition, ignore_duplicates=False,
                                  on_conflict="team_unique, competition_id, league_unique").execute()

In [8]:
add_teams_to_db(team_stats_csv="2024/2024_ncaa_tournament_team_stats.csv", team_seeds_csv="2024/2024_ncaa_tournament_team_seeds.csv", leage_unique="ncaambb", competition_id=5, expected_num_teams=68, thru='2024-03-17', supabase_client=supabase)

In [100]:
# this functionality has been moved into the add_teams_to_db function
def generate_team_seeds(team_seeds_csv, league_unique: str, competition_id: int, expected_num_teams: int):
    team_seed_insert = []
    with open(team_seeds_csv, mode='r', encoding='utf-8-sig') as csv_file:
        seeds_csv_reader = csv.DictReader(csv_file)
        for row in seeds_csv_reader:
            team_seed_row = {key: row[key].lower() if key == 'team_unique' else row[key] for key in row.keys()
                             & {'team_unique', 'seed','overall_seed', 'region'}}
            team_seed_row['competition_id'] = competition_id
            team_seed_row['league_unique'] = league_unique
            team_seed_insert.append(team_seed_row)
    if len(team_seed_insert) != expected_num_teams:
        raise ValueError("Number of teams in seed file does not match expected number of teams")
    return team_seed_insert


def add_seeds_to_db(team_seeds_csv, league_unique: str, competition_id: str, expected_num_teams: int, supabase_client):
    seeds = generate_team_seeds(team_seeds_csv, league_unique, competition_id, expected_num_teams)
    supabase_client.table("team_competition").upsert(seeds, ignore_duplicates=False,
                                  on_conflict="team_unique, competition_id, league_unique").execute()

In [101]:
# add_seeds_to_db(team_seeds_csv="2023_acc_tournament_team_seeds.csv", league_unique="ncaambb", competition_id=3, expected_num_teams=15, supabase_client=supabase)

In [11]:
from datetime import date
def generate_players(player_stats_csv, competition_id: int, league_unique:str, thru=date.today()):
    player_insert = []
    player_competition_insert= []
    player_stat_insert = []
    with open(player_stats_csv, mode='r', encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            player_unique = row['player_id'].lower()
            team_unique=row['team_abbreviation'].lower()
            player_row = {'player_unique': player_unique,'player_name': row['name']}
            if 'birthdate' in row:
                player_row['birthdate'] = row['birthdate']
            if 'position' in row:
                player_row['position'] = row['position']
            player_stats = {key: float(row[key]) if row[key] else 0 for key in row.keys()
                             & {'points', 'assists','blocks', 'minutes_played', 'effective_field_goal_percentage', 'field_goals', 'field_goal_attempts','field_goal_percentage', 'two_pointers', 'two_point_attempts', 'two_point_percentage', 'three_pointers', 'three_point_attempts','three_point_percentage', 'free_throws', 'free_throw_attempts','free_throw_percentage', 'free_throw_attempt_rate', 'offensive_rebounds', 'defensive_rebounds', 'steals', 'turnovers', 'personal_fouls', 'usage_percentage','true_shooting_percentage', 'player_efficiency_rating', 'games_played', 'games_started'}} 
            player_stats['rebounds'] = int(float(row['total_rebounds'])) if row['total_rebounds'] else 0
            player_competition_row = {'player_unique': player_unique, 'competition_id': competition_id,
                                      'league_unique': league_unique, 'team_unique': team_unique, 'inactive': False,
                                      'stats_thru': thru, 'player_stats': player_stats}

            player_insert.append(player_row)
            player_competition_insert.append(player_competition_row)
            player_stat_insert.append(player_stats)
    return [player_insert, player_competition_insert, player_stat_insert]


def add_players_to_db(player_stats_csv,  competition_id: str, league_unique: str, thru, supabase_client):
    [players, player_competition, player_stats] = generate_players(player_stats_csv, competition_id, league_unique, thru)
    supabase.table("player").upsert(players, ignore_duplicates=True,
                                  on_conflict="player_unique").execute()
    supabase.table("player_competition").upsert(player_competition, ignore_duplicates=False, on_conflict='player_unique,competition_id').execute()
    # supabase.table("stats_player_precompetitionsnapshot_basketball").upsert(player_stats, ignore_duplicates=False, on_conflict='player_unique,competition_id').execute()

In [12]:
add_players_to_db(player_stats_csv="2024/2024_ncaa_tournament_player_stats.csv", competition_id=5, league_unique='ncaambb', thru='2023-03-17', supabase_client=supabase)

In [ ]:
add_picks_to_db:

In [13]:
supabase.auth.admin.update_user_by_id({"id": "mitchellbrooks+test7@gmail.com", "password": "assword1" })

TypeError: update_user_by_id() missing 1 required positional argument: 'attributes'